In [3]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 45.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=5e6db2d4ac9dba267530add61e8c5e22fb0f55e43eca0ca259ab1a92e87a81b1
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 40 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Rea

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf



**1. Start a simple Spark Session**









In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()



**2. Load the Walmart Stock CSV File, have Spark infer the data types.**



In [6]:
from google.colab import files
 
 
uploaded = files.upload()

Saving walmart_stock.csv to walmart_stock.csv


In [7]:
import pandas as pd
import io
 
df = pd.read_csv(io.BytesIO(uploaded['walmart_stock.csv']))
print(df)

            Date       Open       High        Low      Close    Volume  \
0     2012-01-03  59.970001  61.060001  59.869999  60.330002  12668800   
1     2012-01-04  60.209999  60.349998  59.470001  59.709999   9593300   
2     2012-01-05  59.349998  59.619999  58.369999  59.419998  12768200   
3     2012-01-06  59.419998  59.450001  58.869999  59.000000   8069400   
4     2012-01-09  59.029999  59.549999  58.919998  59.180000   6679300   
...          ...        ...        ...        ...        ...       ...   
1253  2016-12-23  69.430000  69.750000  69.360001  69.540001   4803900   
1254  2016-12-27  69.300003  69.820000  69.250000  69.699997   4435700   
1255  2016-12-28  69.940002  70.000000  69.260002  69.309998   4875700   
1256  2016-12-29  69.209999  69.519997  69.120003  69.260002   4298400   
1257  2016-12-30  69.120003  69.430000  68.830002  69.120003   6889500   

      Adj Close  
0     52.619235  
1     52.078475  
2     51.825539  
3     51.459220  
4     51.616215  
...

In [8]:
walmart_df = spark.createDataFrame(df)

**3. What are the column names?**

In [9]:
walmart_df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

**4. What does the Schema look like?**

In [10]:
walmart_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: long (nullable = true)
 |-- Adj Close: double (nullable = true)



**5. Print out the first 5 columns.**

In [28]:
walmart_df.show(5)

+----------+---------+---------+---------+---------+--------+---------+
|      Date|     Open|     High|      Low|    Close|  Volume|Adj Close|
+----------+---------+---------+---------+---------+--------+---------+
|2012-01-03|59.970001|61.060001|59.869999|60.330002|12668800|52.619235|
|2012-01-04|60.209999|60.349998|59.470001|59.709999| 9593300|52.078475|
|2012-01-05|59.349998|59.619999|58.369999|59.419998|12768200|51.825539|
|2012-01-06|59.419998|59.450001|58.869999|     59.0| 8069400| 51.45922|
|2012-01-09|59.029999|59.549999|58.919998|    59.18| 6679300|51.616215|
+----------+---------+---------+---------+---------+--------+---------+
only showing top 5 rows



**6. Use describe() to learn about the DataFrame. Provide your inference on the same**

In [30]:
walmart_df.describe().show()

+-------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|             Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|             1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null|72.35785375357703|72.83938807631154|71.91860095945944|72.38844998012719|8222093.481717011| 67.2388384872814|
| stddev|      null|6.768090244708284|6.768186808159222|6.744075756255509|6.756859163732981|  4519780.8431556|6.722609449996835|
|    min|2012-01-03|        56.389999|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|2016-12-30|        90.800003|        90.970001|            89.25|        90.470001|     

**7. From the above question, There may many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns**

In [31]:
from pyspark.sql.functions import format_number # Actually don't need to import as we have import * in the first cell

summary = walmart_df.describe()
summary.select(summary['summary'],
              format_number(summary['Open'].cast('float'), 2).alias('Open'),
              format_number(summary['High'].cast('float'), 2).alias('High'),
              format_number(summary['Low'].cast('float'), 2).alias('Low'),
              format_number(summary['Close'].cast('float'), 2).alias('Close'),
              format_number(summary['Volume'].cast('float'), 2).alias('Volume')).show()

+-------+--------+--------+--------+--------+-------------+
|summary|    Open|    High|     Low|   Close|       Volume|
+-------+--------+--------+--------+--------+-------------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,093.50|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,781.00|
|    min|   56.39|   57.06|   56.30|   56.42| 2,094,900.00|
|    max|   90.80|   90.97|   89.25|   90.47|80,898,096.00|
+-------+--------+--------+--------+--------+-------------+



**8. Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.**

In [32]:
walmart_hv = walmart_df.withColumn('HV Ratio', walmart_df['High']/walmart_df['Volume']).select('HV Ratio')
walmart_hv.show(10)

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001474E-6|
|7.071764823529411...|
+--------------------+
only showing top 10 rows



**9. What day had the Peak High in Price?**

In [35]:
walmart_df.orderBy('High', ascending = False).select(['Date']).show(1)

+----------+
|      Date|
+----------+
|2015-01-13|
+----------+
only showing top 1 row



**10. What is the mean of the Close column?**

In [36]:
walmart_df.agg({'Close' : 'mean'}).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012719|
+-----------------+



**11. What is the max and min of the Volume column?**

In [37]:
walmart_df.agg({'Volume' : 'max'}).show()

+-----------+
|max(Volume)|
+-----------+
|   80898100|
+-----------+



In [38]:
walmart_df.agg({'Volume' : 'min'}).show()

+-----------+
|min(Volume)|
+-----------+
|    2094900|
+-----------+



**12. How many days was the Close lower than 60 dollars?**

In [39]:
walmart_df.filter('Close < 60').count()

81

**13. What percentage of the time was the High greater than 80 dollars ?**

In [40]:
walmart_df.filter('High > 80').count() * 100/walmart_df.count()

9.141494435612083

**14.What is the Pearson correlation between High and Volume?**

In [41]:
walmart_df.corr('High', 'Volume')

-0.3384326061737167

**15.What is the max High per year?**

In [42]:
year_df = walmart_df.withColumn('Year', year(walmart_df['Date']))

In [43]:
year_df.groupBy('Year').agg({'High':'max'}).orderBy('Year').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2012|77.599998|
|2013|81.370003|
|2014|88.089996|
|2015|90.970001|
|2016|75.190002|
+----+---------+



**16.What is the average Close for each Calendar Month?**

In [44]:
month_df = walmart_df.withColumn('Month', month(walmart_df['Date']))

In [45]:
month_df.groupBy('Month').agg({'Close': 'mean'}).orderBy('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|71.30680444329897|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679246|
|    6|72.49537742452831|
|    7|74.43971943925233|
|    8|73.02981855454544|
|    9|72.18411785294116|
|   10|71.57854545454545|
|   11|72.11108930693068|
|   12|72.84792478301885|
+-----+-----------------+

